#### Scraping Audited Contracts from Etherscan, PolygonScan, BSCscan

#### etherscan

In [59]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [60]:
addresses_etherscan=[]
for i in range(1,21):
    url = f"https://etherscan.io/contractsVerified/{i}?filter=audit"
    headers = {
        'User-Agent': '...',
        'referer': 'https://...'
    }
    response = requests.get(url,headers = headers)
    response = response.content
    soup = BeautifulSoup(response, "html.parser")
    table =  soup.find("table")
    spans = table.find_all("span", class_= "d-flex align-items-center gap-1")

    for span in spans:
        link = span.find("a")
        title = link.attrs["title"]
        addresses_etherscan.append(title)


In [61]:
print(len(addresses_etherscan))

500


#### polygonscan

In [62]:

addresses_polygon = []
url = "https://polygonscan.com/contractsVerified/?filter=audit"
headers = {
    'User-Agent': '...',
    'referer': 'https://...'
}
response = requests.get(url,headers = headers)
response = response.content
soup = BeautifulSoup(response, "html.parser")
table =  soup.find("table")
spans = table.find_all("a", class_= "hash-tag text-truncate")
for span in spans:
    title = span.attrs["title"]
    addresses_polygon.append(title)

print(len(addresses_polygon))

24


#### bsc

In [63]:
bsc_addresses = []
for i in range(1,21):
    url = f"https://bscscan.com/contractsVerified/{i}?filter=audit"
    headers = {
        'User-Agent': '...',
        'referer': 'https://...'
    }
    response = requests.get(url,headers = headers)
    response = response.content
    soup = BeautifulSoup(response, "html.parser")
    table =  soup.find("table")
    spans = table.find_all("a", class_= "hash-tag text-truncate")
    for span in spans:
        title = span.attrs["title"]
        bsc_addresses.append(title)

print(len(bsc_addresses))

500


#####  Extracting address from tokens(etherscan) 

In [65]:
token_addresses_etherscan = []
token_name_etherscan = []
token_usd_etherscan = []
for i in range(1,24):
    url = f"https://etherscan.io/tokens?p={i}"
    headers = {
        'User-Agent': '...',
        'referer': 'https://...'
    }
    response = requests.get(url,headers = headers)
    response = response.content
    soup = BeautifulSoup(response, "html.parser")
    table =  soup.find("table")
    
    #to get the usd value of the token
    rows = table.find_all("tr")
    for row in rows[1:]:
        columns = row.find_all("td") #change len(columns)>6
        token_name_etherscan.append(columns[1].find('div', class_= 'hash-tag').text.strip())
        href= (columns[1].find("a", class_= "d-flex align-items-center gap-1 link-dark", href= True)['href'])
        token_addresses_etherscan.append(href.split('/')[-1])
        usd_value = columns[5].text.strip() #circulating marketing cap is in 5th index(td)
        token_usd_etherscan.append(usd_value)


#converting to dataframe
df_etherscan_tok = pd.DataFrame({'Etherscan_Token_Name': token_name_etherscan,
                   'Token_Address': token_addresses_etherscan,
                   'Token_Price': token_usd_etherscan})
print(df_etherscan_tok.shape)
df_etherscan_tok.head(10)

(1144, 3)


,Etherscan_Token_Name,Token_Address,Token_Price
0,Tether USD,0xdac17f958d2ee523a2206206994597c13d831ec7,"$70,424,308,060.00"
1,BNB,0xB8c77482e45F1F44dE1745F52C74426C631bDD52,"$49,857,978,296.00"
2,USD Coin,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,"$41,923,375,487.00"
3,OKB,0x75231f58b43240c9718dd58b4967c5114342a86c,"$13,094,667,620.00"
4,Matic Token,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,"$12,832,299,469.00"
5,Binance USD,0x4fabb145d64652a948d72533023f6e7a623c7c53,"$12,799,256,432.00"
6,HEX,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,"$11,270,777,197.00"
7,stETH,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,"$8,797,551,242.00"
8,SHIBA INU,0x95aD61b0a150d79219dCF64E1E6Cc01f0B64C4cE,"$7,978,255,179.00"
9,ANY Litecoin,0x0abcfbfa8e3fda8b7fba18721caf7d5cf55cf5f5,"$6,922,453,649.00"


#### Scraping BEP_20 Tokens

In [66]:
token_addresses_bsc_bep_20 = []
token_name_bsc = []
token_usd_bsc = []

for i in range(1,17):
    url = f"https://bscscan.com/tokens?p={i}"
    headers = {
        'User-Agent': '...',
        'referer': 'https://...'
    }
    response = requests.get(url,headers = headers)
    response = response.content
    soup = BeautifulSoup(response, "html.parser")
    table =  soup.find("table")
    
    rows = table.find_all("tr")
    for row in rows[1:]:
        columns = row.find_all("td")
        token_name_bsc.append(columns[1].find('a', class_= 'text-primary', href= True).text.strip())
        href= (columns[1].find('a', class_= 'text-primary', href= True)['href'])
        token_addresses_bsc_bep_20.append(href.split('/')[-1])
        usd_value = columns[5].text.strip()
        token_usd_bsc.append(usd_value)
            
df_bsc_tok = pd.DataFrame({'bsc_Token_Name': token_name_bsc,
                   'Token_Address': token_addresses_bsc_bep_20,
                   'Token_Price': token_usd_bsc})

print(df_bsc_tok.shape)
df_bsc_tok.head(10)


(796, 3)


,bsc_Token_Name,Token_Address,Token_Price
0,Binance-Peg Ethereum Token (ETH),0x2170ed0880ac9a755fd29b2688956bd959f933f8,"$208,082,744,875.00"
1,Binance-Peg BSC-USD (BSC-USD),0x55d398326f99059ff775485246999027b3197955,"$70,394,919,623.00"
2,Wrapped BNB (WBNB),0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c,"$49,843,215,439.00"
3,Binance-Peg USD Coin (USDC),0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d,"$41,927,075,666.00"
4,USDC (anyUSDC),0x8965349fb649a33a30cbfda057d8ec2c48abe2a2,"$41,927,075,666.00"
5,Binance-Peg XRP Token (XRP),0x1d2f0da169ceb9fc7b3144628db156f3f6c60dbe,"$20,255,839,831.00"
6,Binance-Peg Cardano Token (ADA),0x3ee2200efb3400fabb9aacf31297cbdd1d435d47,"$13,970,459,392.00"
7,Matic Token (MATIC),0xcc42724c6683b7e57334c4e856f4c9965ed682bd,"$13,218,467,489.00"
8,Binance-Peg BUSD Token (BUSD),0xe9e7cea3dedca5984780bafc599bd69add087d56,"$12,804,495,908.00"
9,Binance-Peg Dogecoin Token (DOGE),0xba2ae424d960c26247dd6c32edc70b295c744c43,"$11,671,069,633.00"


#### Duplicate checking 

In [67]:
def comp(l1, l2):
    same = []
    for val in l1:
        if val in l2:
            same.append(val)
    return same

comp(addresses_etherscan, bsc_addresses)

['0x6c43751fef27c956f7e75d5c345a65df1465f7e0',
 '0x1111111254fb6c44bAC0beD2854e76F90643097d',
 '0x11111112542d85b3ef69ae05771c2dccff4faa26',
 '0x9798dF2f5d213a872c787bD03b2b91F54D0D04A1']

In [68]:
comp(addresses_etherscan, bsc_addresses)

['0x6c43751fef27c956f7e75d5c345a65df1465f7e0',
 '0x1111111254fb6c44bAC0beD2854e76F90643097d',
 '0x11111112542d85b3ef69ae05771c2dccff4faa26',
 '0x9798dF2f5d213a872c787bD03b2b91F54D0D04A1']

In [69]:
unique_bsc_addresses =  [x for x in bsc_addresses if x not in addresses_etherscan]
comp(unique_bsc_addresses, addresses_etherscan)

[]

In [70]:
comp(addresses_etherscan, token_addresses_etherscan)

['0x4fc15c91a9c4a9efb404174464687e8e128730c2',
 '0x3be7bf1a5f23bd8336787d0289b70602f1940875',
 '0xe0c05ec44775e4ad62cdc2eecdf337aa7a143363',
 '0xD2e5decc08A80be6538F89f9AB8ff296e2f724Df',
 '0x5ba19d656b65f1684cfea4af428c23b9f3628f97',
 '0xe0df31d06d72b2f5231489af0edc422b372f49f1',
 '0xb31ef9e52d94d4120eb44fe1ddfde5b4654a6515',
 '0xa5ef74068d04ba0809b7379dd76af5ce34ab7c57',
 '0x9b99cca871be05119b2012fd4474731dd653febe',
 '0x9fa69536d1cda4a04cfb50688294de75b505a9ae',
 '0x41a3dba3d677e573636ba691a70ff2d606c29666',
 '0x50f09629d0afdf40398a3f317cc676ca9132055c',
 '0xaaa7a10a8ee237ea61e8ac46c50a8db8bcc1baaa',
 '0x0c572544a4ee47904d54aaa6a970af96b6f00e1b',
 '0xf6650117017ffd48b725b4ec5a00b414097108a7',
 '0x15b543e986b8c34074dfc9901136d9355a537e7e',
 '0xc834fa996fa3bec7aad3693af486ae53d8aa8b50',
 '0x29cbd0510eec0327992cd6006e63f9fa8e7f33b7',
 '0x14da7b27b2e0fedefe0a664118b0c9bc68e2e9af',
 '0x377d552914e7a104bc22b4f3b6268ddc69615be7',
 '0x0cdf9acd87e940837ff21bb40c9fd55f68bba059',
 '0x69af81e73

In [71]:
addresses_to_remove_from_addresses_etherscan = comp(addresses_etherscan,token_addresses_etherscan)
addresses_to_remove_from_addresses_etherscan

['0x4fc15c91a9c4a9efb404174464687e8e128730c2',
 '0x3be7bf1a5f23bd8336787d0289b70602f1940875',
 '0xe0c05ec44775e4ad62cdc2eecdf337aa7a143363',
 '0xD2e5decc08A80be6538F89f9AB8ff296e2f724Df',
 '0x5ba19d656b65f1684cfea4af428c23b9f3628f97',
 '0xe0df31d06d72b2f5231489af0edc422b372f49f1',
 '0xb31ef9e52d94d4120eb44fe1ddfde5b4654a6515',
 '0xa5ef74068d04ba0809b7379dd76af5ce34ab7c57',
 '0x9b99cca871be05119b2012fd4474731dd653febe',
 '0x9fa69536d1cda4a04cfb50688294de75b505a9ae',
 '0x41a3dba3d677e573636ba691a70ff2d606c29666',
 '0x50f09629d0afdf40398a3f317cc676ca9132055c',
 '0xaaa7a10a8ee237ea61e8ac46c50a8db8bcc1baaa',
 '0x0c572544a4ee47904d54aaa6a970af96b6f00e1b',
 '0xf6650117017ffd48b725b4ec5a00b414097108a7',
 '0x15b543e986b8c34074dfc9901136d9355a537e7e',
 '0xc834fa996fa3bec7aad3693af486ae53d8aa8b50',
 '0x29cbd0510eec0327992cd6006e63f9fa8e7f33b7',
 '0x14da7b27b2e0fedefe0a664118b0c9bc68e2e9af',
 '0x377d552914e7a104bc22b4f3b6268ddc69615be7',
 '0x0cdf9acd87e940837ff21bb40c9fd55f68bba059',
 '0x69af81e73

In [72]:
unique_etherscan_addresses =  [x for x in addresses_etherscan if x not in token_addresses_etherscan]
comp(unique_etherscan_addresses, token_addresses_etherscan)

[]

In [73]:
comp(unique_bsc_addresses,token_addresses_bsc_bep_20)

['0x5711f19b7b21938d31d07e5736b4660c1159d7d3',
 '0x0475AE77fb55005F15554642d6016c75FD04227A',
 '0x9c4a515cd72d27a4710571aca94858a53d9278d5',
 '0xb25583E5e2dB32b7FCbffe3f5e8E305C36157E54',
 '0xC17c30e98541188614dF99239cABD40280810cA3',
 '0x7d38315b92d0E7A85B35B2b7FE969B25935619D7',
 '0x9131066022B909C65eDD1aaf7fF213dACF4E86d0',
 '0x6e4a971b81ca58045a2aa982eaa3d50c4ac38f42',
 '0xce186ad6430e2fe494a22c9edbd4c68794a28b35',
 '0xc75aa1fa199eac5adabc832ea4522cff6dfd521a',
 '0x6cf271270662be1c4fc1b7bb7d7d7fc60cc19125',
 '0xeB84be66c8E71f07eA57Cf3b21626d7784F32A7F',
 '0x9D4451151A8dE5B545a1bC6c8fdEB9d94a2868e1',
 '0x6067490d05f3cf2fdffc0e353b1f5fd6e5ccdf70',
 '0x01E0d17a533E5930A349C2BB71304F04F20AB12B',
 '0x68b5edb385b59e30a7a7db1e681a449e94df0213',
 '0x46d502fac9aea7c5bc7b13c8ec9d02378c33d36f',
 '0x56aa0237244c67b9a854b4efe8479cca0b105289',
 '0xae2df9f730c54400934c06a17462c41c08a06ed8',
 '0x2A9718defF471f3Bb91FA0ECEAB14154F150a385',
 '0xc9849e6fdb743d08faee3e34dd2d1bc69ea11a51']

In [74]:
# List of Token Addresses to be removed
addresses_to_remove_from_addresses_unique_bsc_addresses = comp(unique_bsc_addresses,token_addresses_bsc_bep_20)
addresses_to_remove_from_addresses_unique_bsc_addresses

['0x5711f19b7b21938d31d07e5736b4660c1159d7d3',
 '0x0475AE77fb55005F15554642d6016c75FD04227A',
 '0x9c4a515cd72d27a4710571aca94858a53d9278d5',
 '0xb25583E5e2dB32b7FCbffe3f5e8E305C36157E54',
 '0xC17c30e98541188614dF99239cABD40280810cA3',
 '0x7d38315b92d0E7A85B35B2b7FE969B25935619D7',
 '0x9131066022B909C65eDD1aaf7fF213dACF4E86d0',
 '0x6e4a971b81ca58045a2aa982eaa3d50c4ac38f42',
 '0xce186ad6430e2fe494a22c9edbd4c68794a28b35',
 '0xc75aa1fa199eac5adabc832ea4522cff6dfd521a',
 '0x6cf271270662be1c4fc1b7bb7d7d7fc60cc19125',
 '0xeB84be66c8E71f07eA57Cf3b21626d7784F32A7F',
 '0x9D4451151A8dE5B545a1bC6c8fdEB9d94a2868e1',
 '0x6067490d05f3cf2fdffc0e353b1f5fd6e5ccdf70',
 '0x01E0d17a533E5930A349C2BB71304F04F20AB12B',
 '0x68b5edb385b59e30a7a7db1e681a449e94df0213',
 '0x46d502fac9aea7c5bc7b13c8ec9d02378c33d36f',
 '0x56aa0237244c67b9a854b4efe8479cca0b105289',
 '0xae2df9f730c54400934c06a17462c41c08a06ed8',
 '0x2A9718defF471f3Bb91FA0ECEAB14154F150a385',
 '0xc9849e6fdb743d08faee3e34dd2d1bc69ea11a51']

In [75]:
# unique_etherscan_addresses =  [x for x in addresses_etherscan if x not in token_addresses_etherscan]
# comp(unique_etherscan_addresses, token_addresses_etherscan)
unique_bsc_addresses =  [x for x in unique_bsc_addresses if x not in token_addresses_bsc_bep_20]
comp(unique_bsc_addresses, token_addresses_bsc_bep_20)

[]

In [83]:
print(len(token_addresses_etherscan))
print(len(token_addresses_bsc_bep_20))
print(len(unique_etherscan_addresses))
print(len(unique_bsc_addresses))
print(len(addresses_polygon))

1144
796
461
475
24


In [93]:
df_etherscan_tok['Token_Price']= df_etherscan_tok['Token_Price'].apply(lambda x: x[1:].replace(',', '')).astype(float)

,bsc_Token_Name,Token_Address,Token_Price
0,Binance-Peg Ethereum Token (ETH),0x2170ed0880ac9a755fd29b2688956bd959f933f8,"$208,082,744,875.00"
1,Binance-Peg BSC-USD (BSC-USD),0x55d398326f99059ff775485246999027b3197955,"$70,394,919,623.00"
2,Wrapped BNB (WBNB),0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c,"$49,843,215,439.00"
3,Binance-Peg USD Coin (USDC),0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d,"$41,927,075,666.00"
4,USDC (anyUSDC),0x8965349fb649a33a30cbfda057d8ec2c48abe2a2,"$41,927,075,666.00"
...,...,...,...
791,Wrapped ROSE (Wormhole) (wROSE),0x6c6d604d3f07abe287c1a3df0281e999a83495c0,--
792,Xpose Protocol (XP),0x6a0b045ea1cbc87946ee6ca4d118dc0b5c152455,--
793,xSigma (anySIG),0xc20ff003662d9b91b4139ffa6b7c9d69968bad2c,--
794,YIN Finance (anyYIN),0x0aCAaBC390D7D7134eef4166F61e3e30B362f675,--


In [104]:
df_etherscan_tok.tail()

,Etherscan_Token_Name,Token_Address,Token_Price
1139,Decentralized Accessible Content Chain,0xf8c595d070d104377f58715ce2e6c93e49a87f3c,17.0
1140,Agrolot Token,0x72c9fb7ed19d3ce51cea5c56b3e023cd918baadf,12.0
1141,USDQ Stablecoin by Q DAO v1.0,0x4954db6391f4feb5468b6b943d4935353596aec9,0.0
1142,InterValue,0xdac4ae188ace3c8985765edc6c9b4739d4845ddc,0.0
1143,DGD,0xe0b7927c4af23765cb51314a0e0521a9645f0e2a,0.0


In [97]:
import numpy as np
df_bsc_tok['Token_Price'].replace(['--', np.nan], inplace=True)

In [98]:
df_bsc_tok['Token_Price']= df_bsc_tok['Token_Price'].apply(lambda x: x[1:].replace(',', '')).astype(float)

In [102]:
df_bsc_tok.tail()

,bsc_Token_Name,Token_Address,Token_Price
791,Wrapped ROSE (Wormhole) (wROSE),0x6c6d604d3f07abe287c1a3df0281e999a83495c0,0.0
792,Xpose Protocol (XP),0x6a0b045ea1cbc87946ee6ca4d118dc0b5c152455,0.0
793,xSigma (anySIG),0xc20ff003662d9b91b4139ffa6b7c9d69968bad2c,0.0
794,YIN Finance (anyYIN),0x0aCAaBC390D7D7134eef4166F61e3e30B362f675,0.0
795,Yoshi.exchange (anyYOSHI),0x6c01361d8dbaf43b81b52da3efae7dab1b8fffe9,0.0


In [103]:
df_bsc_tok.loc[df_bsc_tok['Token_Price']!= 0]

,bsc_Token_Name,Token_Address,Token_Price
0,Binance-Peg Ethereum Token (ETH),0x2170ed0880ac9a755fd29b2688956bd959f933f8,2.080827e+11
1,Binance-Peg BSC-USD (BSC-USD),0x55d398326f99059ff775485246999027b3197955,7.039492e+10
2,Wrapped BNB (WBNB),0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c,4.984322e+10
3,Binance-Peg USD Coin (USDC),0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d,4.192708e+10
4,USDC (anyUSDC),0x8965349fb649a33a30cbfda057d8ec2c48abe2a2,4.192708e+10
...,...,...,...
401,MMToken (MM),0x73f059db0182dc2ed716d028e1b13033cfcbf63b,1.380400e+04
402,Fusible | Fusible.io (FUSII),0x3a50d6daacc82f17a2434184fe904fc45542a734,1.207100e+04
403,Multiplier (bMXX),0x4131b87f74415190425ccd873048c708f8005823,6.111000e+03
404,SF Capital (SFCP),0x5ce8e55e256e4bd58cea94963d4cb280de6b8bb2,5.799000e+03


In [121]:
df_eth_fin= df_etherscan_tok.loc[df_etherscan_tok['Token_Price']>5*1e5]

In [122]:
df_bsc_fin= df_bsc_tok.loc[df_bsc_tok['Token_Price']> 1e6]

In [123]:
print(len(unique_etherscan_addresses)
+(len(unique_bsc_addresses))
+(len(addresses_polygon))
+df_eth_fin.shape[0]
+df_bsc_fin.shape[0])

2081


# The Total Smart Contracts (Audited+Tokens) from Etherscan, PolygonScan, BSCScan are- 2081